##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Object Detection with TensorFlow Lite Model Maker

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/lite/tutorials/model_maker_object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/tutorials/model_maker_object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/tutorials/model_maker_object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/tutorials/model_maker_object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In this colab notebook, I will use other notebook as an example. It uses [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) library to train a custom object detection model capable of detecting salads within images on a mobile device.

My dataset is located on my GoogleDrive, contains classes
* big (truck)
* trailerbig (truck with a trailer)

These pictures were collected by me, with RPi3B and Motion app running.



## Prerequisites


### Mount Google drive with pics and labels

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library you'll use for evaluation.

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [18]:
!pip install -q tensorflow==2.5.0
!pip install -q tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 454.3 MB 19 kB/s 
     |████████████████████████████████| 462 kB 31.7 MB/s 
     |████████████████████████████████| 1.2 MB 33.9 MB/s 
     |████████████████████████████████| 4.0 MB 35.7 MB/s 


After this please restart your runtime! To make Tensorflow 2.5 version loaded.

Import the required packages.

In [10]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2.5')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from PIL import Image

### Prepare the dataset

Dataset contains PascalVoc annotations and images in the same folder

## Quickstart

In [11]:
IMAGES_AND_LABELS_FOLDER = "/content/gdrive/MyDrive/Colab Notebooks/watching_trucks"

In [12]:
import glob
xmls = glob.glob(os.path.join(IMAGES_AND_LABELS_FOLDER, "*.xml"))
len(xmls)

105

In [13]:
import glob
jpgs = glob.glob(os.path.join(IMAGES_AND_LABELS_FOLDER, "*.jpg"))
print(len(jpgs))
assert len(jpgs) == len(xmls)

105


In [14]:
import random
import math
import os

random.seed(100)

def train_valid_test_split(path, split_ratio=(0.8,0.1,0.1)):  
  all_images = list(os.path.basename(filename) for filename in glob.glob(os.path.join(path, "*.jpg")))
  if len(split_ratio) != 3:
    raise AttributeError("you should provide a tuple with 3 fractions for split- train,valid,test")
  if sum(split_ratio) != 1:
    raise AttributeError("Split should add up to 1.0")
  train_len = math.floor(split_ratio[0] * len(all_images))
  train_images = random.sample(all_images, train_len)
  other_images = list(set(all_images) - set(train_images))
  valid_len = math.floor(split_ratio[1] * len(all_images))
  valid_images = random.sample(other_images, valid_len)
  test_images = list(set(other_images) - set(valid_images))
  return train_images, valid_images, test_images

train_images, valid_images, test_images = train_valid_test_split(IMAGES_AND_LABELS_FOLDER)

print("train has %d elements, valid %d, test %d" % (len(train_images), len(valid_images), len(test_images)))
print(train_images[:5], valid_images[:5])

train has 84 elements, valid 10, test 11
['17-20210809112610-01.jpg', '30-20210809124649-00.jpg', '64-20210809164944-01.jpg', '64-20210809164737-00.jpg', '17-20210809112729-00.jpg'] ['64-20210809164500-00.jpg', '18-20210809113245-01.jpg', '17-20210809112638-01.jpg', '64-20210809164139-00.jpg', '64-20210809164733-00.jpg']


In [15]:
# just a sanity check if we really have 3 separate sets of images

assert set(test_images).intersection(set(train_images)) == set()
assert set(valid_images).intersection(set(train_images)) == set()
assert set(test_images).intersection(set(valid_images)) == set()

In [16]:
train_names = [filename.replace(".jpg", "") for filename in train_images]
valid_names = [filename.replace(".jpg", "") for filename in valid_images]
test_names =  [filename.replace(".jpg", "") for filename in test_images]

There are six steps to training an object detection model:

**Step 1. Choose an object detection model archiecture.**

This tutorial uses the EfficientDet-Lite0 model. EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 37            | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 49            | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 69            | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 116           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 260           | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Pixel 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>


In [17]:
spec = model_spec.get('efficientdet_lite1')

**Step 2. Load the dataset.**

Model Maker will take input data in the CSV format. Use the `object_detector.DataLoader.from_csv` method to load the dataset and split them into the training, validation and test images.

* Training images: These images are used to train the object detection model to recognize salad ingredients.
* Validation images: These are images that the model didn't see during the training process. You'll use them to decide when you should stop the training, to avoid [overfitting](https://en.wikipedia.org/wiki/Overfitting).
* Test images: These images are used to evaluate the final model performance.

You can load the CSV file directly from Google Cloud Storage, but you don't need to keep your images on Google Cloud to use Model Maker. You can specify a local CSV file on your computer, and Model Maker will work just fine.

In [18]:
images_path = IMAGES_AND_LABELS_FOLDER
annotations_path = IMAGES_AND_LABELS_FOLDER

label_map = {1:"big", 2:"bigtrailer"}

train_dataloader = object_detector.DataLoader.from_pascal_voc(images_path, annotations_path, label_map, annotation_filenames=train_names)
valid_dataloader = object_detector.DataLoader.from_pascal_voc(images_path, annotations_path, label_map, annotation_filenames=valid_names)
test_dataloader = object_detector.DataLoader.from_pascal_voc(images_path, annotations_path, label_map, annotation_filenames=test_names)

**Step 3. Train the TensorFlow model with the training data.**

* The EfficientDet-Lite0 model uses `epochs = 50` by default, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop early to avoid overfitting.
* Set `batch_size = 8` here so you will see that it takes 21 steps to go through the 175 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [19]:
EPOCHS = 120
model = object_detector.create(train_dataloader, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=valid_dataloader, epochs=EPOCHS)

Epoch 1/120
10/10 [==============================] - 131s 9s/step - det_loss: 1.8070 - cls_loss: 1.1500 - box_loss: 0.0131 - reg_l2_loss: 0.0706 - loss: 1.8776 - learning_rate: 0.0090 - gradient_norm: 1.5645 - val_det_loss: 1.6278 - val_cls_loss: 1.1188 - val_box_loss: 0.0102 - val_reg_l2_loss: 0.0706 - val_loss: 1.6984
Epoch 2/120
10/10 [==============================] - 81s 8s/step - det_loss: 1.6645 - cls_loss: 1.0861 - box_loss: 0.0116 - reg_l2_loss: 0.0706 - loss: 1.7351 - learning_rate: 0.0100 - gradient_norm: 2.1427 - val_det_loss: 1.3506 - val_cls_loss: 0.9581 - val_box_loss: 0.0079 - val_reg_l2_loss: 0.0706 - val_loss: 1.4212
Epoch 3/120
10/10 [==============================] - 80s 8s/step - det_loss: 1.3507 - cls_loss: 0.8786 - box_loss: 0.0094 - reg_l2_loss: 0.0706 - loss: 1.4213 - learning_rate: 0.0100 - gradient_norm: 3.1711 - val_det_loss: 1.3174 - val_cls_loss: 1.0345 - val_box_loss: 0.0057 - val_reg_l2_loss: 0.0706 - val_loss: 1.3879
Epoch 4/120
10/10 [=================

KeyboardInterrupt: ignored

**Step 4. Evaluate the model with the test data.**

After training the object detection model using the images in the training dataset, use the remaining 25 images in the test dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 step to go through the 25 images in the test dataset.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [20]:
model.evaluate(test_dataloader)

NameError: ignored

**Step 5.  Export as a TensorFlow Lite model.**

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is full integer quantization.

In [ ]:
model.export(export_dir='.')

**Step 6.  Evaluate the TensorFlow Lite model.**

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
model.evaluate_tflite('model.tflite', test_dataloader)

You can download the TensorFlow Lite model file using the left sidebar of Colab. Right-click on the `model.tflite` file and choose `Download` to download it to your local computer.

This model can be integrated into an Android or an iOS app using the [ObjectDetector API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) of the [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview).

See the [TFLite Object Detection sample app](https://github.com/tensorflow/examples/blob/master/lite/examples/object_detection/android/lib_task_api/src/main/java/org/tensorflow/lite/examples/detection/tflite/TFLiteObjectDetectionAPIModel.java#L91) for more details on how the model is used in an working app.

*Note: Android Studio Model Binding does not support object detection yet so please use the TensorFlow Lite Task Library.*

## (Optional) Test the TFLite model on your image

You can test the trained TFLite model using images from the internet.
* Replace the `INPUT_IMAGE_URL` below with your desired input image.
* Adjust the `DETECTION_THRESHOLD` to change the sensitivity of the model. A lower threshold means the model will pickup more objects but there will also be more false detection. Meanwhile, a higher threshold means the model will only pickup objects that it has confidently detected.

Although it requires some of boilerplate code to run the model in Python at this moment, integrating the model into a mobile app only requires a few lines of code.

In [ ]:
#@title Load the trained TFLite model and define some visualization functions

import cv2

from PIL import Image

model_path = 'model.tflite'

# Load the labels into a list
classes = ['???'] * model.model_spec.config.num_classes
label_map = model.model_spec.config.label_map
for label_id, label_name in label_map.as_dict().items():
  classes[label_id-1] = label_name

# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  return resized_img, original_image


def set_input_tensor(interpreter, image):
  """Set the input tensor."""
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image


def get_output_tensor(interpreter, index):
  """Retur the output tensor at the given index."""
  output_details = interpreter.get_output_details()[index]
  tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
  return tensor


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""
  # Feed the input image to the model
  set_input_tensor(interpreter, image)
  interpreter.invoke()

  # Get all outputs from the model
  boxes = get_output_tensor(interpreter, 0)
  classes = get_output_tensor(interpreter, 1)
  scores = get_output_tensor(interpreter, 2)
  count = int(get_output_tensor(interpreter, 3))

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
testing_model_image_path = "/content/gdrive/MyDrive/Colab Notebooks/watching_trucks/test_new"

In [ ]:
otherimages = os.listdir(testing_model_image_path)
otherimages[:10]

In [ ]:
model_path = 'model.tflite'
#@title Run object detection and show the detection results

FILENAME = "IMG_0968.JPEG" #@param {type:"string"}
DETECTION_THRESHOLD = 0.3 #@param {type:"number"}
folder = testing_model_image_path
image_path = os.path.join(folder, FILENAME)

im = Image.open(image_path)
im.thumbnail((512, 512), Image.ANTIALIAS)

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    image_path,
    interpreter,
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
Image.fromarray(detection_result_image)